In [1]:
from google.colab import files

# Mengunggah file pickle dari komputer lokal ke Colab
uploaded = files.upload()

Saving svd_model.pkl to svd_model.pkl


In [30]:
# Instal library Surprise
!pip install scikit-surprise

In [31]:


# Import library yang dibutuhkan
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from datetime import datetime

# Memuat model SVD yang sudah disimpan sebelumnya
with open('svd_model.pkl', 'rb') as f:
    model = pickle.load(f)

# === Load Dataset ===
df_tourism_with_id = pd.read_csv('/content/drive/MyDrive/archive (1)/tourism_with_id.csv')  # Path ke tourism_with_id
df_tourism_rating = pd.read_csv('/content/drive/MyDrive/archive (1)/tourism_rating.csv')    # Path ke tourism_rating

# Gabungkan dataset untuk akses lebih mudah
df_merged = pd.merge(df_tourism_rating, df_tourism_with_id, on='Place_Id', how='inner')

# Tambahkan simulasi timestamp jika diperlukan
df_merged['Timestamp'] = pd.date_range(start='2023-01-01', periods=len(df_merged), freq='D')

# Preprocessing data untuk library Surprise
reader = Reader(rating_scale=(0, 5))  # Rating berkisar antara 0 hingga 5
data = Dataset.load_from_df(df_tourism_rating[['User_Id', 'Place_Id', 'Place_Ratings']], reader)

# Membagi dataset menjadi data latih dan uji
trainset, testset = train_test_split(data, test_size=0.2)

# Menggunakan model yang dimuat untuk pengujian
predictions = loaded_model.test(testset)

# Evaluasi dengan RMSE
rmse_value = rmse(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse_value}")


RMSE: 1.1095
Root Mean Squared Error (RMSE): 1.109450706190896


In [19]:
def rekomendasi_pengguna_baru(kategori, lokasi=None, top_n=5):
    """
    Rekomendasi tempat wisata untuk pengguna baru berdasarkan kategori wisata dan lokasi.
    """
    # Filter berdasarkan kategori
    filtered_places = df_merged[df_merged['Category'].str.contains(kategori, case=False)]

    # Filter berdasarkan lokasi jika disediakan
    if lokasi:
        filtered_places = filtered_places[filtered_places['City'].str.contains(lokasi, case=False)]

    # Urutkan berdasarkan rating rata-rata tempat wisata
    rekomendasi = (
        filtered_places[['Place_Name', 'Category', 'City', 'Rating']]
        .drop_duplicates()
        .sort_values(by='Rating', ascending=False)
        .head(top_n)
    )
    return rekomendasi

In [20]:
# === Tambahkan Informasi Tren ===
# Hitung jumlah kunjungan untuk setiap tempat
trend_visits = df_merged.groupby('Place_Id').size().reset_index(name='Visit_Count')

# Hitung rata-rata rating terkini (6 bulan terakhir)
cutoff_date = datetime.now() - pd.Timedelta(days=180)
recent_ratings = df_merged[df_merged['Timestamp'] > cutoff_date]
trend_ratings = recent_ratings.groupby('Place_Id')['Place_Ratings'].mean().reset_index(name='Recent_Avg_Rating')

# Gabungkan informasi tren dengan dataset wisata
df_tourism_with_id = df_tourism_with_id.merge(trend_visits, on='Place_Id', how='left').merge(trend_ratings, on='Place_Id', how='left')
df_tourism_with_id['Visit_Count'] = df_tourism_with_id['Visit_Count'].fillna(0)
df_tourism_with_id['Recent_Avg_Rating'] = df_tourism_with_id['Recent_Avg_Rating'].fillna(0)

In [39]:
# Memuat model SVD yang sudah disimpan sebelumnya
with open('svd_model.pkl', 'rb') as f:
    model = pickle.load(f)

def rekomendasi_pengguna_lama(user_id, top_n=5):
    """
    Memberikan rekomendasi untuk pengguna lama berdasarkan model SVD yang telah dilatih dan interaksi pengguna lain yang serupa.
    """
    # Ambil semua Place_Id
    all_places = df_tourism_rating['Place_Id'].unique()

    # Tempat yang sudah pernah dilihat oleh user
    seen_places = df_tourism_rating[df_tourism_rating['User_Id'] == user_id]['Place_Id'].unique()

    # Prediksi rating untuk tempat yang belum dilihat
    rekomendasi = []
    for place_id in all_places:
        if place_id not in seen_places:
            pred = model.predict(user_id, place_id)
            rekomendasi.append((place_id, pred.est))

    # Urutkan berdasarkan rating prediksi tertinggi
    rekomendasi = sorted(rekomendasi, key=lambda x: x[1], reverse=True)[:top_n]

    # Ambil detail tempat wisata yang direkomendasikan
    rekomendasi_df = pd.DataFrame(rekomendasi, columns=['Place_Id', 'Predicted_Rating'])
    rekomendasi_df = rekomendasi_df.merge(df_tourism_with_id[['Place_Id', 'Place_Name', 'Category']], on='Place_Id')

    return rekomendasi_df[['Place_Id', 'Place_Name', 'Category', 'Predicted_Rating']]

In [22]:
# === Contoh Penggunaan ===
# Rekomendasi untuk pengguna baru
kategori_wisata = "Budaya"
lokasi_wisata = "Yogyakarta"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                        Place_Name Category        City  Rating
582   Kauman Pakualaman Yogyakarta   Budaya  Yogyakarta     5.0
1323          Museum Ullen Sentalu   Budaya  Yogyakarta     4.7
446                Candi Borobudur   Budaya  Yogyakarta     4.7
314            Wisata Kraton Jogja   Budaya  Yogyakarta     4.7
163                Candi Prambanan   Budaya  Yogyakarta     4.7


In [23]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan lokasi
kategori_wisata = "Budaya"
lokasi_wisata = "Semarang"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                     Place_Name Category      City  Rating
94              Pura Giri Natha   Budaya  Semarang     4.7
141          Tugu Muda Semarang   Budaya  Semarang     4.7
527          Kota Lama Semarang   Budaya  Semarang     4.7
977  Kampoeng Djadhoel Semarang   Budaya  Semarang     4.7
739                 Lawang Sewu   Budaya  Semarang     4.6


In [24]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan tanpa lokasi
kategori_wisata = "Budaya"
lokasi_wisata = ""
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                          Place_Name Category        City  Rating
582     Kauman Pakualaman Yogyakarta   Budaya  Yogyakarta     5.0
1232                 Freedom Library   Budaya     Jakarta     5.0
324            Galeri Indonesia Kaya   Budaya     Jakarta     4.8
155   Museum Sasmita Loka Ahmad Yani   Budaya     Jakarta     4.7
314              Wisata Kraton Jogja   Budaya  Yogyakarta     4.7


In [25]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan tanpa lokasi
kategori_wisata = "Alam"
lokasi_wisata = ""
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                       Place_Name    Category      City  Rating
653  Kebun Tanaman Obat Sari Alam  Cagar Alam   Bandung     4.9
171                Gunung Lalakon  Cagar Alam   Bandung     4.8
732      Gua Maria Kerep Ambarawa  Cagar Alam  Semarang     4.8
520                 Curug Aseupan  Cagar Alam   Bandung     4.7
58                       Wot Batu  Cagar Alam   Bandung     4.7


In [26]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan lokasi
kategori_wisata = "Alam"
lokasi_wisata = "Yogyakarta"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                                Place_Name    Category        City  Rating
725            Bukit Paralayang, Watugupit  Cagar Alam  Yogyakarta     4.7
337  Puncak Gunung Api Purba - Nglanggeran  Cagar Alam  Yogyakarta     4.7
11                      Taman Sungai Mudal  Cagar Alam  Yogyakarta     4.6
890                           Goa Jomblang  Cagar Alam  Yogyakarta     4.6
135              Seribu Batu Songgo Langit  Cagar Alam  Yogyakarta     4.6


In [41]:
# Rekomendasi untuk pengguna lama
user_id = 1
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=1:
   Place_Id                Place_Name       Category  Predicted_Rating
0        14             Pulau Pelangi         Bahari          4.177344
1         1          Monumen Nasional         Budaya          4.057149
2       224            Dago Dreampark  Taman Hiburan          4.047253
3       332            Rainbow Garden     Cagar Alam          4.019562
4       291  Water Park Bandung Indah  Taman Hiburan          4.000103


In [40]:
# Rekomendasi untuk pengguna lama
user_id = 2
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=2:
   Place_Id                             Place_Name       Category  \
0       416                       Keraton Surabaya         Budaya   
1       254                 Teras Cikapundung BBWS  Taman Hiburan   
2       139  Puncak Gunung Api Purba - Nglanggeran     Cagar Alam   
3       134                   Desa Wisata Gamplong  Taman Hiburan   
4       401                         Taman Keputran  Taman Hiburan   

   Predicted_Rating  
0          3.755080  
1          3.565000  
2          3.548573  
3          3.543048  
4          3.531473  


In [42]:
# Rekomendasi untuk pengguna lama
user_id = 3
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=3:
   Place_Id                             Place_Name    Category  \
0       416                       Keraton Surabaya      Budaya   
1        52                           Kampung Cina      Budaya   
2       139  Puncak Gunung Api Purba - Nglanggeran  Cagar Alam   
3         1                       Monumen Nasional      Budaya   
4       122                            Watu Goyang      Budaya   

   Predicted_Rating  
0          4.327892  
1          4.278960  
2          4.123304  
3          4.051696  
4          4.041698  
